In [1]:
import os
from dotenv import load_dotenv
load_dotenv()
import warnings
warnings.filterwarnings('ignore')

# 필수 라이브러리 로드
from langchain_openai import OpenAIEmbeddings
from langchain_chroma import Chroma
from langchain_core.documents import Document
from langchain_text_splitters import RecursiveCharacterTextSplitter
import time
import pickle

# 이전 단계 데이터 로드
print('청크데이터 로드')
chunks_file = 'chunks_output_4_2_RAG2.pkl'

if os.path.exists(chunks_file):
    with open(chunks_file, 'rb') as f:   # 바이너리 형식..? 불러올때 지정??? 
        doc_chunks = pickle.load(f)
else:
    # 파일이 없으면 새로 생성
    print('4_2_RAG1.2_chunk.ipynb 실행')

# openai 임베딩 모델 초기화
embeddng_model = OpenAIEmbeddings(
    model = 'text-embedding-3-small'
)

# 단일 텍스트 임베딩 테스트
test_text = 'RAG는 검색 증강 생성 기술입니다.'

# 시간비교
start_time = time.time()

# 텍스트 쿼리 임베딩
test_embedding = embeddng_model.embed_query(test_text)

# 시간비교
elasped = time.time() - start_time

print(f'입력텍스트 : {test_text}')
print(f'벡터차원: {len(test_embedding)}')
print(f'벡터일부: {test_embedding[0]:.4f} {test_embedding[1]:.4f} ...{test_embedding[-1]:.4f}')
print(f'소요시간: {elasped:.3f}')


청크데이터 로드
입력텍스트 : RAG는 검색 증강 생성 기술입니다.
벡터차원: 1536
벡터일부: 0.0114 0.0539 ...-0.0661
소요시간: 0.657


In [2]:

# 유사도 계산
import numpy as np
def cosine_similarity(vec1, vec2):
    dot_product = np.dot(vec1, vec2)
    norm1 = np.linalg.norm(vec1)
    norm2 = np.linalg.norm(vec2)
    return dot_product / (norm1*norm2)

test_sentences = [
    "RAG는 검색 증강 생성 기술입니다.",         # 기준 문장
    "RAG는 문서 검색과 답변 생성을 결합합니다.",  # 유사한 문장
    "벡터 데이터베이스는 임베딩을 저장합니다.",   # 관련 있는 문장
    "오늘 날씨가 매우 좋습니다.",               # 관련 없는 문장
]

# 모든 문장을 임베딩
embeddings = [embeddng_model.embed_query(sent) for sent in test_sentences]

# 기존 문장과 유사도 비교
base_embedding = embeddings[0]

print(f'기준문장 : {test_sentences[0]}')
print(f'유사도 비교 결과')
for i, (sent, emb) in enumerate(zip(test_sentences[1:], embeddings[1:]), 1) :
    similarity= cosine_similarity(base_embedding, emb)
    print(f'문서 {i}번째 : {sent} ---> {similarity:.4f}')

기준문장 : RAG는 검색 증강 생성 기술입니다.
유사도 비교 결과
문서 1번째 : RAG는 문서 검색과 답변 생성을 결합합니다. ---> 0.6779
문서 2번째 : 벡터 데이터베이스는 임베딩을 저장합니다. ---> 0.2335
문서 3번째 : 오늘 날씨가 매우 좋습니다. ---> 0.0664


In [3]:
# chromaDB (벡터DB)
# 동작방식
# 저장 : 텍스트(청크) --> 임베딩 (벡터) --> VectorDB(저장)
# 검색 : 질문 --> 임베딩(벡터) --> 유사도 검색 --> top-k문서 변환
embeddng_model = OpenAIEmbeddings(
    model = 'text-embedding-3-small'
)
# chromaDB에 chunk를 저장
start_time = time.time()


# chromaDB 생성(인메모리 방식)
chunks_file = 'chunks_output_4_2_RAG2.pkl'

if os.path.exists(chunks_file):
    with open(chunks_file, 'rb') as f:   # 바이너리 형식..? 불러올때 지정??? 
        doc_chunks = pickle.load(f)
else:
    # 파일이 없으면 새로 생성
    print('4_2_RAG1.2_chunk.ipynb 실행')


vectorstore = Chroma.from_documents(
    documents = doc_chunks,
    collection_name = '4_2_RAG1.2_chunk', 
    embedding=embeddng_model
)

elasped = time.time() - start_time
print(f'VectorDB 구축완료')
print(f'저장된 청크수 : {len(doc_chunks)}')
print(f'소요시간: {elasped}')



VectorDB 구축완료
저장된 청크수 : 6
소요시간: 1.2081890106201172


In [28]:
# 테스트 질문
test_queries = [
    'RAG란 무엇인가요?',
    'VectorDB에는 어떤 종류가 있나요?',
    'LangChain의 구성 요소는?'
]

for j, query in enumerate(test_queries, 1):
    print(f'\n\n질문 {j} : {query}')

    # 유사문서 검색 상위 2개
    results = vectorstore.similarity_search_with_score(query, k=2)
    for i, (doc,score) in enumerate(results, 1):
        source = doc.metadata.get('source', 'unknown')
        preview = doc.page_content.strip()[:].replace('\n', ' ')
        print(f'-----> 답변{i} {source} (거리: {score:.4f}') # 적을수록 좋음. (거리반환이므로, 거리 가까울수록 값 적게나옴)
        print(f' {preview}')




질문 1 : RAG란 무엇인가요?
-----> 답변1 rag_concept.txt (거리: 0.8739
 RAG (Retrieval-Augmented Generation)는 검색 증강 생성 기술입니다.          RAG의 작동 원리:         1. 사용자 질문을 임베딩 벡터로 변환합니다.         2. 벡터 데이터베이스에서 유사한 문서를 검색합니다.         3. 검색된 문서를 컨텍스트로 사용하여 LLM이 답변을 생성합니다.          RAG의 장점:         - 최신 정보를 반영할 수 있습니다. LLM의 학습 데이터 이후 정보도 활용 가능합니다.
-----> 답변2 rag_concept.txt (거리: 0.9984
 - 환각(Hallucination)을 감소시킵니다. 실제 문서 기반으로 답변하기 때문입니다.         - 출처를 명시할 수 있습니다. 어떤 문서에서 정보를 가져왔는지 추적 가능합니다.         - 도메인 특화가 가능합니다. 특정 분야의 문서만 사용하여 전문적인 답변을 제공합니다.          RAG의 핵심 구성요소: Retriever(검색기), Generator(생성기), VectorStore(벡터저장소)


질문 2 : VectorDB에는 어떤 종류가 있나요?
-----> 답변1 vectordb_intro.txt (거리: 0.7170
 VectorDB(벡터 데이터베이스)는 고차원 벡터를 효율적으로 저장하고 검색하는 데이터베이스입니다.          주요 VectorDB 솔루션:         - ChromaDB: 로컬 개발에 적합한 오픈소스 솔루션. 파이썬 네이티브로 설치가 간편합니다.         - Pinecone: 완전 관리형 클라우드 서비스. 대규모 프로덕션 환경에 적합합니다.         - Weaviate: 그래프 기반 벡터 데이터베이스. 하이브리드 검색을 지원합니다.
-----> 답변2 vectordb_intro.txt (거리: 1.3665
 - FAISS: Facebook에서

In [33]:
# 다양한 검색 옵션
print('다양한 검색 옵션')
# 리트리버 생성
print('기본 유사도 검색 (similarity)')
retriver_basic = vectorstore.as_retriever(   #청킨된 다큐먼트 벡터를 가지고 있는게 vectorstore
                search_type = 'similarity'   #retriver similarity 이용?
                , search_kwargs = {'k': 3}
) 
results = retriver_basic.invoke('RAG의 장점')
print(f'결과 : {len(results)}')
for i, doc in enumerate(results, 1):
    print(f"{i} : {doc.metadata.get('source', 'unknown')}")

다양한 검색 옵션
기본 유사도 검색 (similarity)
결과 : 3
1 : rag_concept.txt
2 : rag_concept.txt
3 : vectordb_intro.txt


In [ ]:
print(f'MMR 검색 (다양성 고려)')
retriver_basic = vectorstore.as_retriever(   #청킨된 다큐먼트 벡터를 가지고 있는게 vectorstore
                search_type = 'mmr'   #mmr 이용
                , search_kwargs = {'k': 3,
                                   'fetch_k': 6,  #먼저 6개의 후보 검색
                                   'lambda' : 0.5  # 다양성 가중치 (0=다양성, 1=관련성)
                                   }  
) 
results = retriver_basic.invoke('RAG의 장점')
print(f'결과 : {len(results)}')
for i, doc in enumerate(results, 1):
    print(f"{i} : {doc.metadata.get('source', 'unknown')}") # source, unkown?

MMR 검색 (다양성 고려)
결과 : 3
1 : rag_concept.txt
2 : langchain_intro.txt
3 : vectordb_intro.txt


In [35]:
print(f'metadata를 필터링')  # 필터링 하면 조건지정이 가능?
results = vectorstore.similarity_search(
    '기술에 대해 설명해 주세요',
    k=2,
    filter={'topic': 'technique'} # 주제가 technique한 것을.. 
)
print(f'결과 : {len(results)}')
for i, doc in enumerate(results, 1):
    print(f"{i} : {doc.metadata.get('source', 'unknown')} topic = {doc.metadata.get('topic')}")


metadata를 필터링
결과 : 2
1 : rag_concept.txt topic = technique
2 : rag_concept.txt topic = technique


In [38]:
#vectorDB 영구 저장 (옵션임/ 필수아님)
persist_dir = './chroma_db_reg2'
vectorstore_persistent = Chroma.from_documents(
    documents= doc_chunks,
    collection_name='persistent_rag',
    embedding= embeddng_model,
    persist_directory=persist_dir
)

print(f'vectorDB 영구저장')
print(f'저장경로 : {persist_dir}')
print(f'저장된 청크수 : {len(doc_chunks)}')

# 설정정보 저장
# json 저장해도 되나, json 보다 pickle로 하면 용량이 작아지고 장점이 있음 / json은 딕셔너리만 가능
config = {
    'persist_directory': persist_dir,
    'collection_name' : "persistent_rag",
    'embedding_model': 'text-embedding-3-small',
    'chunk_count' : len(doc_chunks) 
}
with open('vectordb_config_4_4_RAG2.2.pkl', 'wb') as f:
    pickle.dump(config, f)

print('설정정보 저장 완료 파일명 : vectordb_config_4_4_RAG2.2.pkl')


vectorDB 영구저장
저장경로 : ./chroma_db_reg2
저장된 청크수 : 6
설정정보 저장 완료 파일명 : vectordb_config_4_4_RAG2.2.pkl
